# Examen Práctico – Redes Neuronales  
## Implementación de un Perceptrón para Regresión  


En este notebook se desarrolla paso a paso la implementación de un perceptrón simple para un problema de regresión, utilizando el conjunto de datos **California Housing**.

El objetivo es conectar de manera explícita la teoría presentada en el libro (conceptos como vector de características, pesos, sesgo, suma ponderada y
función de pérdida) con una implementación práctica en Python, empleando principalmente **NumPy** y herramientas básicas de **scikit-learn** para la
carga y normalización de los datos.

El notebook funciona como un cuaderno de trabajo experimental, donde se implementan y prueban los distintos componentes del modelo, mientras que las
justificaciones teóricas detalladas se presentan por separado en el documento `respuestas_teoricas.md`.

En esta sección se carga el conjunto de datos California Housing utilizando
`fetch_california_housing` de sklearn. Se extraen las características de entrada
y la variable objetivo, y se imprime una descripción general del dataset para
entender su estructura.

In [47]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar el dataset
california = fetch_california_housing()
X = california.data
y = california.target
feature_names = california.feature_names

print('Descripción del dataset:')
print(california.DESCR[:1500])

Descripción del dataset:
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. cen

El conjunto de datos se divide en subconjuntos de entrenamiento y prueba.
Posteriormente, las características se normalizan usando `StandardScaler`
para garantizar una escala uniforme, lo cual es importante para el entrenamiento
estable del perceptrón.

In [48]:
# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Normalización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("X_train_scaled shape:", X_train_scaled.shape)


X_train_scaled shape: (16512, 8)


Se inicializan los parámetros del perceptrón: el vector de pesos `w` y el sesgo `b`.
Los pesos se inicializan con valores aleatorios pequeños y el sesgo en cero,
siguiendo las recomendaciones teóricas del modelo del perceptrón.

In [49]:
import numpy as np

def inicializar_parametros(n_caracteristicas):
    '''
    Inicializa los pesos y el sesgo del perceptrón.
    '''
    w = np.random.randn(n_caracteristicas, 1) * 0.01
    b = 0.0
    return w, b


In [50]:
w, b = inicializar_parametros(X_train_scaled.shape[1])
print("Forma de w:", w.shape)
print("Sesgo b:", b)

Forma de w: (8, 1)
Sesgo b: 0.0


### Propagación hacia adelante (suma ponderada)

En esta sección se implementa la propagación hacia adelante del perceptrón.
Siguiendo el Capítulo 2 del material del curso, se calcula la suma ponderada:

\[
z = X \mathbf{w} + b
\]

donde \(X\) es la matriz de características, \(\mathbf{w}\) el vector de pesos y \(b\) el sesgo.
En un problema de regresión, esta salida corresponde directamente a la predicción del modelo.


In [51]:
def propagacion_adelante(X, w, b):
    '''
    Calcula la suma ponderada para todas las observaciones.
    '''
    y_pred = X @ w + b
    return y_pred

In [52]:
# Predicciones iniciales para los primeros 5 ejemplos
y_pred_5 = propagacion_adelante(X_train_scaled[:5], w, b)

print("Predicciones iniciales:")
print(y_pred_5.flatten())

print("\nValores reales:")
print(y_train[:5])

Predicciones iniciales:
[-0.02219977 -0.00341057  0.00708927 -0.01912173  0.00092429]

Valores reales:
[1.03  3.821 1.726 0.934 0.965]


Las predicciones iniciales no son precisas, ya que el modelo aún no ha sido entrenado.
Los pesos se inicializaron con valores aleatorios pequeños, por lo que la salida inicial
del modelo es cercana a cero.

### Función de pérdida: Error Cuadrático Medio (MSE)

En problemas de regresión, la calidad de las predicciones se evalúa mediante una función de pérdida.
En este trabajo se utiliza el **Error Cuadrático Medio (MSE)**, que mide el promedio del cuadrado de las diferencias entre las predicciones del modelo y los valores reales.


In [53]:
def calcular_perdida(y_pred, y_real):
    '''
    Calcula el error cuadrático medio (MSE).

    MSE = (1/m) * sum((y_pred - y_real)^2)
    '''
    m = y_real.shape[0]
    mse = (1 / m) * np.sum((y_pred - y_real) ** 2)
    return mse

In [54]:
# Calcular predicciones iniciales
y_pred_inicial = propagacion_adelante(X_train_scaled, w, b)

# Calcular pérdida inicial
perdida_inicial = calcular_perdida(y_pred_inicial, y_train.reshape(-1, 1))

perdida_inicial

np.float64(5.61312792620843)

## Cálculo de gradientes

En esta sección se implementa el cálculo de los gradientes de la función de pérdida
(MSE) respecto a los pesos \(w\) y al sesgo \(b\), siguiendo las expresiones
derivadas en la Parte III del reporte teórico.

In [55]:
# Propagación hacia adelante
y_pred = propagacion_adelante(X_train_scaled, w, b)

print("Forma de y_pred:", y_pred.shape)

Forma de y_pred: (16512, 1)


In [56]:
def calcular_gradientes(X, y_pred, y_real):
    '''
    Calcula los gradientes de la pérdida respecto a w y b.
    
    Parámetros:
    -----------
    X : numpy array de forma (m, n)
    y_pred : numpy array de forma (m, 1)
    y_real : numpy array de forma (m, 1)
    
    Retorna:
    --------
    dw : numpy array de forma (n, 1) - gradiente respecto a w
    db : float - gradiente respecto a b
    '''
    m = X.shape[0]
    
    # Error de predicción
    error = y_pred - y_real.reshape(-1, 1)
    
    # Gradientes
    dw = (2 / m) * X.T @ error
    db = (2 / m) * np.sum(error)
    
    return dw, db

In [57]:
dw, db = calcular_gradientes(X_train_scaled, y_pred, y_train)

print("Forma de w:", w.shape)
print("Forma de dw:", dw.shape)
print("Gradiente db:", db)

Forma de w: (8, 1)
Forma de dw: (8, 1)
Gradiente db: -4.143893874757745


## Actualización de parámetros

En esta sección se implementa la regla de actualización de los pesos y el sesgo
mediante descenso del gradiente, con el objetivo de reducir la función de pérdida.

In [58]:
def actualizar_parametros(w, b, dw, db, learning_rate):
    '''
    Actualiza los pesos y el sesgo usando gradiente descendente.
    
    w_nuevo = w - learning_rate * dw
    b_nuevo = b - learning_rate * db
    '''
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b


### Una iteración del descenso del gradiente

A continuación se ejecuta una iteración completa del algoritmo de aprendizaje:
propagación hacia adelante, cálculo de la pérdida, cálculo de gradientes y
actualización de los parámetros.


In [59]:
# Pérdida antes de la actualización
y_pred = propagacion_adelante(X_train_scaled, w, b)
loss_before = calcular_perdida(y_pred, y_train)

# Cálculo de gradientes
dw, db = calcular_gradientes(X_train_scaled, y_pred, y_train)

# Actualización de parámetros
learning_rate = 0.01
w, b = actualizar_parametros(w, b, dw, db, learning_rate)

# Pérdida después de la actualización
y_pred_new = propagacion_adelante(X_train_scaled, w, b)
loss_after = calcular_perdida(y_pred_new, y_train)

print("Pérdida antes:", loss_before)
print("Pérdida después:", loss_after)

Pérdida antes: 92964.52718416127
Pérdida después: 90167.58732431363
